## Consultas a la base de datos

In [2]:
!pip install pymysql

In [1]:
import pymysql

In [2]:
class DataBase:
    def __init__(self):
        self.connection = pymysql.connect(
            host = "localhost",
            user = "root",
            password = "",
            db = "teleton"
        )
        self.cursor = self.connection.cursor()
    
        print("Conexión exitosa")
    
    def insert_users(self, nomina, nombre, puesto, tags, certificado, estatus):
            sql = f"INSERT INTO users (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES ('{nomina}', '{nombre}', '{puesto}', '{tags}', '{certificado}', '{estatus}')"
            try:
                self.cursor.execute(sql)
                print(f"{nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def insert_documentos(self, Hash, Tipo, Nombre, Descripcion, Tags, TIME, Estatus):
            sql = f"INSERT INTO documentos (`Hash`, `Tipo`, `Nombre`, `Descripcion`, `Tags`, `TIME`, `Estatus`) VALUES ('{Hash}', '{Tipo}', '{Nombre}', '{Descripcion}', '{Tags}', '{TIME}', '{Estatus}')"
            try:
                self.cursor.execute(sql)
                print(f"{Nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def insert_firma(self, Hash, Doc_signed, TIME, Nomina): 
            Doc_signed = open(Doc_signed, "rb").read()
            sql = """ INSERT INTO firmas  (Hash, Doc_signed, TIME, Nomina) VALUES (%s,%s,%s,%s)"""
            try:
                self.cursor.execute(sql, (Hash, Doc_signed, TIME, Nomina))
                print(f"{Hash} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def select(self, tabla, what, where, value):
        if what == "*":
            sql = f"SELECT {what} FROM {tabla}"
        else:
            sql = f"SELECT {what} FROM {tabla} where {where} = '{value}'"
        self.cursor.execute(sql)
        val = self.cursor.fetchall()
        return val
        
    def update(self, tabla, what, value, where_col, where_val):
        sql = f"UPDATE {tabla} set {what} = '{value}' where {where_col} = '{where_val}'"
        self.cursor.execute(sql)
        print("Actualización exitosa")
        
    def delete(self, tabla, where, value):
        sql = f"DELETE FROM {tabla} where {where} = '{value}'"
        self.cursor.execute(sql)
        print("Eliminación exitosa")

database = DataBase()

Conexión exitosa


### Insertar un usuario

In [3]:
database.insert_users(nomina = "A01752080",
                nombre = "Jairo",
                puesto = "Estudiante",
                tags = "IDM",
                certificado = "pdjsnem",
                estatus = "Activo")

Jairo ha sido añadido a la base de datos.


### Insertar un documento

In [4]:
database.insert_documentos(Hash = "jxajdljdljswss", 
                           Tipo = "Certificado", 
                           Nombre = "Ventas.pdf", 
                           Descripcion = "Documento que bla bla",
                           Tags = "Ventas", 
                           TIME = "Ahorita", 
                           Estatus = "Activo")

Ventas.pdf ha sido añadido a la base de datos.


### Insertar una firma

In [42]:
database.insert_firma(Hash = "A01752468",
                Doc_signed = "nombreDoc_nomina_fecha20220419_110201.sign",
                TIME = "Estudiante",
                Nomina = "IDM")

A01752468 ha sido añadido a la base de datos.


### Ejemplo de select

In [13]:
#"""
database.select(tabla = "documentos", 
                 what = "*", 
                where = "ID", 
                value = "1")
#"""

(('jxajdljdljs',
  'Certificado',
  'Ventas.pdf',
  'Documento que bla bla',
  'Ventas',
  'Ahorita',
  'Activo'),
 ('jxajdljdljswss',
  'Certificado',
  'Ventas.pdf',
  'Documento que bla bla',
  'Ventas',
  'Ahorita',
  'Activo'))

In [50]:
F = database.select(tabla = "firmas", 
                what = "ID, Nomina, Doc_signed", 
                where = "Hash", 
                value = "A01752468")
F#[1][2]

((7,
  'IDM',
  b'p\' |t\xa8\x04U\xb7\xb1\xa88\xb4cK#CK\xd4#P\x14\xc7j\xe2\xd7o\xb2\xab1\r\xb9L\xea\x19\x85 \x98<\r\x86w(L\xb8\xba\xf8\x87\xa1|(\x14<{\xf2\x81\xeb\x1e\xbdl\x03\x8d\x05C\xf9\xa0a\xdf\xc2-\x1b\xb5\xda-\xbfl\x12\xad\x8f\xdbG#9X@\x99tr\xa0R\x8e\xf3E\xeaK\xe9\xc4\x99\xe7\x89\x03\x9c\x11\x88\xa0b\x9e\x14(\xaa\x11\x1b`\r\xdc1\x13`\xb9\xfc7\x89_\xac\x8dA[\xe3\x08\x04t\xea\x01\x90\x8c\xfe)g\x8ap\xf6W3\xf9a\xee\x7f\xcc\xbb\xe4D\xa6\xc3\xf5RHGI`Ty\xbb\xd0g\xf1\xa8\xf5A\x95.w#\xfe\xf7\x8d\xc1(6\x16\x99X\x08\xeb\x03\x1f\xe9h\xe4\x99>xr\x9b\x11\x8e\xb1\xbd\xf1\x96\x88R\x98\xdf8\xed \x82\xc9\x87dp&\x7f.v6\xec\xea\x94\xcf\xe9\xf9/\xbd;\xcb\xdb3\x8fi\xfe\x81\x9e\x8c\xdf"^Y\x80\xd5C\xe4\'\xc1\xd6\x96\xa2\xb7\xe1B\xbd\xe3\xaeo\x8b\'q\xd5sv\x85\xef\x0e\xb0\xa7\xcc\x98zS\x97\x0b"@\xbd(\x1duf\x05n\x89{\xdc\xba\xd2\x08\x1e\x85\xa9\x91\x9c\xc2\xee\x89\xa5\x01J\xee\x85\xfes\xc2L?=\xf5\xe2\xda\xdbs\x82\xd7\xf1\t\xc9:k\xcb\x98\x9e)o$.j\xa6\x04UQ\x019\x02\xf9\xa6\xf0\x92\x14\x15\xd86@\x131\xd2\xf5

### Update

In [55]:
database.update(tabla = "users", 
                what = "Nombre", 
                value="Juanito", 
                where_col = "Nomina", 
                where_val = "A01752068")

Actualización exitosa


### Delete

In [32]:
database.delete(tabla = "firmas", 
                where = "ID", 
                value="2")

Eliminación exitosa


## Generación del certificado digital y llave privada

In [8]:
from OpenSSL import crypto, SSL

def cert_gen(
    emailAddress="emailAddress",
    commonName="commonName",
    countryName="NT",
    localityName="localityName",
    stateOrProvinceName="stateOrProvinceName",
    organizationName="organizationName",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds=0,
    validityEndInSeconds=10*365*24*60*60,
    KEY_FILE = "private.key",
    CERT_FILE="selfsigned.crt"):
    #can look at generated file using openssl:
    #openssl x509 -inform pem -in selfsigned.crt -noout -text
    # create a key pair
    k = crypto.PKey()
    k.generate_key(crypto.TYPE_RSA, 4096)
    # create a self-signed cert
    cert = crypto.X509()
    cert.get_subject().C = countryName
    cert.get_subject().ST = stateOrProvinceName
    cert.get_subject().L = localityName
    cert.get_subject().O = organizationName
    cert.get_subject().OU = organizationUnitName
    cert.get_subject().CN = commonName
    cert.get_subject().emailAddress = emailAddress
    cert.set_serial_number(serialNumber)
    cert.gmtime_adj_notBefore(0)
    cert.gmtime_adj_notAfter(validityEndInSeconds)
    cert.set_issuer(cert.get_subject())
    cert.set_pubkey(k)
    cert.sign(k, 'sha256')
    with open(CERT_FILE, "wt") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert).decode("utf-8"))
    with open(KEY_FILE, "wt") as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, k).decode("utf-8"))

In [9]:
cert_gen(emailAddress="A01750443@tec.mx",
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "privateJERS_2.key",
    CERT_FILE="CertificadoJERS_2.crt")

## Cargar el certificado

In [30]:
import OpenSSL.crypto 
cert = OpenSSL.crypto.load_certificate(
    OpenSSL.crypto.FILETYPE_PEM, 
    open('CertificadoJERS_2.crt').read()
)
cert.get_notBefore()

b'20220418135149Z'

### Obteniendo valores del certificado

In [31]:
print(cert.get_signature_algorithm()) #Con qué algoritmo fue firmado
print(cert.get_notAfter()) #fecha de expiración
print(cert.get_pubkey())
print(cert.get_subject()) # Datos sobre el certificado

b'sha256WithRSAEncryption'
b'20230418135149Z'
<X509Name object '/C=MX/ST=Nuevo Le\xC3\xB3n/L=Monterrey/O=Tecnol\xC3\xB3gico de Monterrey/OU=organizationUnitName/CN=tec.mx/emailAddress=A01750443@tec.mx'>


## Revisar si un certificado coincide con la llave privada

In [33]:
def check_associate_cert_with_private_key(cert, private_key):
    """
    :type cert: str
    :type private_key: str
    :rtype: bool
    """
    try:
        private_key_obj = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, open(private_key).read())
    except OpenSSL.crypto.Error:
        raise Exception('private key is not correct: %s' % private_key)

    try:
        cert_obj = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, open(cert).read())
    except OpenSSL.crypto.Error:
        raise Exception('certificate is not correct: %s' % cert)

    context = OpenSSL.SSL.Context(OpenSSL.SSL.TLSv1_METHOD)
    context.use_privatekey(private_key_obj)
    context.use_certificate(cert_obj)
    try:
        context.check_privatekey()
        return True
    except OpenSSL.SSL.Error:
        return False

In [35]:
check_associate_cert_with_private_key("CertificadoJERS.crt", "privateJERS.key")

True

## Hash de cualquier documento

In [34]:
import sys
import hashlib

# BUF_SIZE is totally arbitrary, change for your app!
BUF_SIZE = 65536  # lets read stuff in 64kb chunks!

md5 = hashlib.md5()
sha1 = hashlib.sha256()
#hashlib.sha512

with open("../Ex.pdf", 'rb') as f:
    while True:
        data = f.read(BUF_SIZE)
        if not data:
            break
        md5.update(data)
        sha1.update(data)

print("MD5: {0}".format(md5.hexdigest()))
print("SHA1: {0}".format(sha1.hexdigest()))

MD5: cb59cfe68bdea88095422f5ef154150b
SHA1: b977d0cddabbddd755654139072285aff38f918ef2f41acefe9742f7580c4877


## Generando y verificando un certificado

In [38]:
import signVerify

file_name = signVerify.gen_signature("privateJERS.key", bytes(sha1.hexdigest(), 'utf-8'))

In [39]:
#file_name = "nombreDoc_nomina_fecha20220419_102737.sign"
signVerify.verify("certificadoJERS.crt", bytes(sha1.hexdigest(), 'utf-8'), file_name)

Verification successful


In [40]:
signVerify.verify("certificadoJERS_2.crt", bytes(sha1.hexdigest(), 'utf-8'), file_name)

Verification failed


In [51]:
signVerify.verify("certificadoJERS.crt", bytes(sha1.hexdigest(), 'utf-8'), F[0][2], load = False)

Verification successful


# Ejemplo de uso del sistema